# Logistic Regression Using Retail Relay Data

This notebook will demonstrate predicting customer retention using logistic regression.

Assignment Questions:

1. Use the Relay data to develop a model to predict customer retention. You may use logistic regression to predict the variable “retained.” You can use any combination of the independent variables available in the data to obtain a model
with the best predictive ability and usability. You are free to use different transformations and combinations of the independent variables. Be aware that there is no “magic bullet” to finding the ideal model. You will have to go through multiple iterations.

2. Split the data into test and train. Generally, 70% of the data is used to train the model and a hold out sample of 30% is used to test the accuracy of the model.

3. Create the first model with all the variables that can be used in the given data set. Interpret the coefficients in a real-world sense.

4. Once you obtain the best model that you can find, predict retention in the test data. You will use the logistic regression coefficients obtained from the train data to do this. This new variable will be the predicted value of retained.

5. Calculate the hit rate. This can be calculated as % of matches between the value of the variable “retained” and the predicted value of “retained” in the test data.


In [1]:
import os
import pandas as pd 
import numpy as np 
from sklearn import preprocessing
import matplotlib.pyplot as plt 
from statsmodels.graphics.mosaicplot import mosaic
plt.rc('font', size = 14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns 
sns.set(style = 'white')
sns.set(style = 'whitegrid', color_codes = True)
#import warnings
#warnings.filterwarnings('ignore', category = FutureWarning)

## Read Data

In [ ]:
fname = "Relay_Data.xlsx"
df = pd.read_excel(fname, sheet_name=2)

In [2]:
# top 5 observations
df.head()

NameError: name 'df' is not defined

In [ ]:
# bottom 5 observations
df.tail()

### 

In [ ]:
df.shape

In [ ]:
# columns and their data types
df.dtypes

### The dataset contains 30801 observations of 15 variables
* The target variable is called 'retained'
* There are several variables that may not be useful to include in the regression model itself such as:
    * custid (ID), created (Datetime), firstorder (Datetime), lastorder (Datetime)
    * firstorder and lastorder should be Datetime format, but they come encoded as 'object' type.
    * Although these may not end up in the regression model, it might be good to explore the data visually using these variables in conjunction with the target variable 'retained'. 
* Some preprocessing and exploration is definitely in scope given the data.
* There are two 'object' type variables 'favday' and 'city', which may be useful to include in the regression model. These should be one-hot-encoded to create dummy variables for modeling.

## Data Dictionary
* **custid**:       Computer generated ID to identify customers throughout the database
* **retained**:     1, if customer is assumed to be active, 0 = otherwise
* **created**:	    Date when the contact was created in the database - when the customer joined
* **firstorder**:	Date when the customer placed first order
* **lastorder**:	Date when the customer placed last order
* **esent**:    	Number of emails sent
* **eopenrate**:	Number of emails opened divided by number of emails sent
* **eclickrate**:	Number of emails clicked divided by number of emails sent
* **avgorder**: 	Average order size for the customer
* **ordfreq**:  	Number of orders divided by customer tenure
* **paperless**:	1 if customer subscribed for paperless communication (only online)
* **refill**:   	1 if customer subscribed for automatic refill
* **doorstep**: 	1 if customer subscribed for doorstep delivery
* **favday**:   	Customer's favorite delivery day
* **city**:     	City where the customer resides in


## Validate 'custid' is unique

In [ ]:
print("There are {:d} total observations".format(df.shape[0]))

In [ ]:
print("There are {:d} unique customer ids".format(len(set(df.custid))))

### Customer ID is in fact not the unique identifier of this dataset

## Check for Missing Data

In [ ]:
# count of missing values by column
df.isna().sum()

### There are exactly 20 missing values in each of these variables:
* 'custid','created','firstorder','lastorder'
* The following observations contain missing values in these variables
* For the purposes of this demonstration, these observations will still be used for logistic regression modeling as the target variable and proposed independent variables do not contain any missing values.

In [ ]:
missing_data = df[df.isnull().any(axis = 1)]
missing_data.shape

## Check for duplicate observations

In [ ]:
df = df.reset_index()
df.loc[df.duplicated(keep = False)]

### The dataset does not contain any duplicated observations

## Count of unique values by variable

In [ ]:
[col + ' ' + str(df[col].nunique()) for col in list(df.columns)]

### The following independent variables are categorical in nature:
* 'paperless' - binary
* 'refill 2' - binary
* 'doorstep' - binary
* 'favday' - categorical 7-levels
* 'city' - categorical 4-levels

## Check for Class Imbalance in Target Variable 'retained'

In [ ]:
df['retained'].value_counts()

In [ ]:
24472 / (24472+6329)

### There is a class imbalance in the Target Variable 'retained'. 80% of observations are 'retained' = 1; 20% of observations are 'retained' = 0. Due to the class imbalance, it will be necessary to perform resampling to balance the classes prior to modeling.

## Explore Average Values by Class
* For each target class label, compute the average value of each independent variable

In [ ]:
df.columns

In [ ]:
# variables for modeling
these_vars = ['retained','esent','eopenrate','eclickrate','avgorder','ordfreq']
df[these_vars].groupby('retained').mean()

### On average:
* 'esent' (number of emails sent to customer) is substantially higher for retained customers than for non-retained customers.
    * **Retained customers were sent 34 emails on average versus 4 emails for non-retained customers**
* 'eopenrate' (emails opened) and 'eclickrate' (emails clicked) are slightly higher as well for retained customers than for non-retained customers.
* 'avgorder' (average order size) is similar for retained customers and non-retained customers. This doesn't appear to be a very distinguishing factor.
* 'ordfreq' (ratio of number of orders to customer tenure) is also similar between retained and non-retained customers.


## Visualize Distributions of Variables by Retained and Non-Retained Customers
* For each of the variables ('esent','eopenrate','eclickrate','avgorder','ordfreq') assess how different the distributions are for retained vs. non-retained customers

### Number of Emails Sent by 'retained'

In [ ]:
sns.boxplot(y = 'esent', x = 'retained', data = df, palette='colorblind')

### From the visualization, there does appear to be a significant difference in the number of emails sent to the customer between classes.

### Number of Emails Opened by 'retained'

In [ ]:
sns.boxplot(y = 'eopenrate', x = 'retained', data = df, palette = 'colorblind')

### From the visualization, there does appear to be a significant difference in the number of emails opened between classes, but also exhibits a lot of uncertainty due to the overlapping interquartile ranges of the distributions.

### Number of Emails Clicked by 'retained'

In [ ]:
sns.boxplot(y = 'eclickrate', x = 'retained', data = df, palette = 'colorblind')

### From the visualization, there does appear to be a significant difference in the number of emails clicked between classes.

### Average Order Size by 'retained'

In [ ]:
sns.boxplot(y = 'avgorder', x = 'retained', data = df, palette = 'colorblind')

### From the visualization, there doesn't appear to be a significant difference in average order size between classes.

### Order Frequency by 'retained'

In [ ]:
sns.boxplot(y = 'ordfreq', x = 'retained', data = df, palette = 'colorblind')

### From the visualization, there doesn't appear to be a significant difference in order frequency between classes.

## Confirm differences between classes using T-Test

In [ ]:
from scipy import stats
varlist = ['esent','eopenrate','eclickrate','avgorder','ordfreq']
for var in varlist:
    t, p = stats.ttest_ind(
        df.loc[df.retained == 0, var],
        df.loc[df.retained == 1, var]
    )
    print("variable:", var)
    print("t statistic:", t)
    print("p-value:", p)

### The t-tests show that the mean differences in the variables 'esent', 'eopenrate', and 'eclickrate' are significantly different from zero between retained and non-retained customers. 
* Although the p-values show statistical significance in each case, there is a large contrast between the t statistic for 'esent' compared to the others. 
* This may informally indicate that 'esent' is the biggest factor in distinguising retained versus non-retained.

## Create a new variable 'orddiff'
* This is the difference in days between first order and last order date
* This may be a proxy variable for customer tenure, which isn't available in this dataset.

In [ ]:
# difference in days between first order and last order date
orddiff = pd.to_datetime(df['lastorder'], errors = 'coerce') - pd.to_datetime(df['firstorder'], errors = 'coerce')
df['orddiff'] = orddiff

In [ ]:
# orddiff summary for whole dataset
df.orddiff.describe()

In [ ]:
# orddiff summary for non-retained customers
df.loc[df.retained == 0, 'orddiff'].describe()

In [ ]:
# orddiff summary for retained customers
df.loc[df.retained == 1, 'orddiff'].describe()

In [ ]:
t,p = stats.ttest_ind(
    df.loc[df.retained == 0, 'orddiff'].values.astype(np.int32),
    df.loc[df.retained == 1, 'orddiff'].values.astype(np.int32)
)
print(t, p)

### There is not a huge difference between retained and non-retained customers concerning the variable 'orddiff' (proxy for *tenure*), but it appears that retained customers had on average more days between their first order and last order.
* T-test shows that the mean difference between classes are not significantly different from zero.

## Compare categorical variables: 'paperless' vs 'retained'
* 1 if customer subscribed for paperless communication (only online)

In [ ]:
from statsmodels.graphics.mosaicplot import mosaic
import pylab
# categorical independent variables are:
# ('paperless','refill','doorstep','favday','city')

# function to generate categorical analysis
def getcat(lst):
    myvars = lst
    mosaic(df, myvars)
    pylab.show()
    print(pd.crosstab(df[myvars[0]], df[myvars[1]]))

In [ ]:
getcat(['paperless','retained'])

### The majority of retained customers subscribed for paperless communication (only online), however for non-retained customers, the majority did not subscribe to paperless communication. This variable appears to distinguish the classes fairly well.

## Compare categorical variables: 'refill' vs 'retained'
* 1 if customer subscribed for automatic refill

In [ ]:
getcat(['refill','retained'])

### The majority of retained customers did not subscribe for automatic refill. This variable doesn't appear to distinguish the classes very well.

## Compare categorical variables: 'doorstep' vs 'retained'
* 1 if customer subscribed for doorstep delivery

In [ ]:
getcat(['doorstep','retained'])

### The majority of both retained and non-retained customers did not subscribe for doorstep delivery. This variable doesn't appear to distinguish the classes very well.

## Compare categorical variables: 'favday' vs 'retained'
* Customer's favorite delivery day

In [ ]:
getcat(['favday','retained'])

### Mondays and Tuesdays were the most preferred days for both retained and non-retained customers. Weekends were the least preferred days for customers. This variable doesn't appear to distinguish the classes very well.

## Compare categorical variables: 'city' vs 'retained'
* City where the customer resides in

In [ ]:
getcat(['city','retained'])

### This variable doesn't appear to distinguish the classes very well as the proportions by city do not contast much.

## Final Variables used for modeling
* For simplicity, the target variable 'retained' will be renamed to 'y'

In [ ]:
# rename class label variable to 'y'
df = df.rename(columns = {'retained': 'y'}) # rename column
# columns to keep
keep_cols = ['y','esent','eopenrate','eclickrate','avgorder','ordfreq','paperless']
data_final = df[keep_cols]
data_final.columns.values

## Check for Multi-Collinearity

### Variance Inflation Factors

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# independent variables
X = data_final.loc[:, data_final.columns != 'y']
variables = list(range(X.shape[1]))
vif = [variance_inflation_factor(X.iloc[:, variables].values, ix) for ix in range(X.iloc[:, variables].shape[1])]
pd.DataFrame({'variable': X.columns, 'vif': vif})

### Correlation matrix

In [ ]:
X.corr()

### There exists some moderate correlation between the independent variables, but all appear to be within acceptable ranges for modeling.

## Over-sampling using SMOTE
* up-sample the non-retained observations using the SMOTE algorithm(Synthetic Minority Oversampling Technique). At a high level, SMOTE:

    * Works by creating synthetic samples from the minor class (complex parts) instead of creating copies.
    * Randomly choosing one of the k-nearest-neighbors and using it to create a similar, but randomly tweaked, new observations.

In [ ]:
from imblearn.over_sampling import SMOTE
# independent variables
X = data_final.loc[:, data_final.columns != 'y']
# target variable
y = data_final.loc[:, data_final.columns == 'y'].values.ravel()
# set seed for over-sampling
os = SMOTE(random_state=0) 
# train-test split; test set size = 30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
columns = X_train.columns
os_data_X, os_data_y = os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data = os_data_X, columns = columns)
os_data_y = pd.DataFrame(data = os_data_y, columns = ['y'])
# check results from over-sampling
print("length of oversampled data is ",len(os_data_X))
print("Number of non-retained customers in oversampled data",len(os_data_y[os_data_y['y']==0]))
print("Number of retained customers in oversampled data",len(os_data_y[os_data_y['y']==1]))
print("Proportion of non-retained customers in oversampled data is ",len(os_data_y[os_data_y['y']==0])/len(os_data_X))
print("Proportion of retained customers in oversampled data is ",len(os_data_y[os_data_y['y']==1])/len(os_data_X))

## Write Over-Sampled Data to File
* save training data

In [50]:
dat = pd.concat([os_data_X, os_data_y], axis=1)
dat.to_csv('data.csv')

## Fit a Logistic Regression Model (sklearn)

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size = 0.3, random_state = 0)
logreg = LogisticRegression() # negate regularization effect that sklearn applies to logistic regression
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

## Predicting the test set results and calculating the accuracy

In [52]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.89


In [53]:
logreg.intercept_

array([-3.57134328])

In [54]:
logreg.coef_

array([[ 0.1835137 ,  0.0063645 ,  0.01378084, -0.00329958, -0.06070313,
         0.47667634]])

In [55]:
# write training and test data to csv
dat = X_train
dat['y'] = y_train
dat.to_csv("train.csv")
dat = X_test
dat['y'] = y_test
dat.to_csv("test.csv")